★ 전체 코드 구조 
** 해당 코드는 모델을 여러 번 평가해서, 과적합 없이 신뢰할 수 있는 성능 점수를 얻기 위해 만든 도구 **

● 라이브러리 가져오기 필요한 기능을 불러옴 
  (numpy, sklearn, defaultdict, deepcopy 등)

● 함수 1: calculate_classification_metrics
 👉 예측 결과와 실제 값을 비교해서 "정확도(ACC), 정밀도(PREC), 재현율(REC), F1 점수, MCC" 계산

● 함수 2: k_fold_model_evaluation
👉 모델을 K-Fold 교차검증, 검증 세트 & 테스트 세트 모두에서 성능 지표 평가

In [1]:
from collections import defaultdict
from copy import deepcopy

In [3]:
import numpy as np

In [5]:
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics

● calculate_classification_metrics(y_preds, y_trues) :
✅ 모델이 예측한 결과(y_preds)와 실제 정답(y_trues)을 비교해서
    성능 평가 점수(Accuracy, Precision, Recall, F1, MCC)를 계산

In [11]:
def calculate_classification_metrics(y_preds, y_trues):
    y_preds = np.round(y_preds)
    y_trues = np.round(y_trues)
    return {
        "ACC": metrics.accuracy_score(y_trues, y_preds),
        "PREC": metrics.precision_score(y_trues, y_preds, average="binary"),
        "REC": metrics.recall_score(y_trues, y_preds, average="binary"),
        "F1": metrics.f1_score(y_trues, y_preds, average="binary"),
        "MCC": metrics.matthews_corrcoef(y_trues, y_preds)
    }

● k_fold_model_evaluation :  
✅ 
모델을 여러 번 훈련하고 평가, 검증 결과 & 테스트 결과 종합평가
K-Fold 교차검증 + 최종 테스트 성능 함께 봄

In [15]:
def k_fold_model_evaluation( model_func, model_parameters, x_train, y_train, x_test, y_test,
     fit_parameters={}, n_splits=10, shuffle=True, random_state=0 ):
     
    valid_eval = defaultdict(list)
    test_eval = defaultdict(list)
    
# K-Fold 나누기 ; 데이터셋 n_splits (기본 10개) 나누고, 훈련/검증 셋 번갈아 구성
    k_fold = StratifiedKFold( 
        n_splits=n_splits,
        shuffle=shuffle,
        random_state=random_state)

    models = [ ]

In [21]:
def k_fold_model_evaluation( model_func, model_parameters, x_train, y_train, x_test, y_test,
     fit_parameters={}, n_splits=10, shuffle=True, random_state=0 ):
     
    valid_eval = defaultdict(list)
    test_eval = defaultdict(list)

    k_fold = StratifiedKFold(
        n_splits=n_splits,
        shuffle=shuffle,
        random_state=random_state)

    models = [ ]

    for train_index, fold_index in k_fold.split(np.zeros(len(x_train)), y_train.ravel()):
        
        model = model_func(**model_parameters)
        
        x_fold_train, x_fold_test = x_train.iloc[train_index, :], x_train.iloc[fold_index, :]
        y_fold_train, y_fold_test = y_train.iloc[train_index], y_train.iloc[fold_index]
        
        try:
            model = model.fit( x_fold_train, y_fold_train, eval_set=[(x_fold_test, y_fold_test)], eval_metric="aucpr", **fit_parameters )
        except:
            model = model.fit( x_fold_train, y_fold_train, **fit_parameters )

        models.append( deepcopy(model) )

        y_pred = model.predict_proba( x_fold_test )[:, 1]
        
        for metric, value in calculate_classification_metrics(list(y_pred), list(y_fold_test)).items():
            valid_eval[metric].append(value)

    for model in models:
        y_test_preds = model.predict_proba( x_test )[:, 1]
        for metric, value in calculate_classification_metrics(y_test_preds, y_test).items():
            test_eval[metric].append(value)

    return valid_eval, test_eval  
 
    # valid_eval: 각 Fold 검증 결과 
    # test_eval: 각 모델이 테스트 데이터에 대해 낸 결과